In [103]:
## what effect does time have on location, district, race, age 

In [104]:
import pandas as pd

def standardize_item_no(df):
    df.loc[:, "item_number"] = df.item_number.str.lower().str.strip().str.replace(r"-", "", regex=False)
    return df

def clean():
    df1 = pd.read_csv("../data/electronic_police_report_2018.csv")
    df2 = pd.read_csv("../data/electronic_police_report_2019.csv")
    df3 = pd.read_csv("../data/electronic_police_report_2020.csv")
    df4 = pd.read_csv("../data/electronic_police_report_2021.csv")
    df5 = pd.read_csv("../data/electronic_police_report_2022.csv")
    df1["year"] = 2018
    df2["year"] = 2019
    df3["year"] = 2020
    df4["year"] = 2021
    df5["year"] = 2022
    dfs = [df1, df2, df3, df4, df5]
    dfa = pd.concat(dfs, join="outer")
    dfa = dfa.pipe(standardize_item_no)

    dfb = pd.read_csv("../data/rtcc.csv")
    dfb["footage_requested"] = "1"
    dfb = dfb.pipe(standardize_item_no)

    df = pd.merge(dfa, dfb, on="item_number")
    return df

In [105]:
df = clean()

C:\Users\PC\AppData\Local\Temp\ipykernel_20232\1662552057.py:11: DtypeWarning: Columns (21) have mixed types. Specify dtype option on import or set low_memory=False.
  df4 = pd.read_csv("../data/electronic_police_report_2021.csv")
C:\Users\PC\AppData\Local\Temp\ipykernel_20232\1662552057.py:12: DtypeWarning: Columns (21) have mixed types. Specify dtype option on import or set low_memory=False.
  df5 = pd.read_csv("../data/electronic_police_report_2022.csv")


In [106]:
df.item_number.unique()

array(['d2077118', 'e1274818', 'l2194418', ..., 'j2225922', 'a1349122',
       'a3158222'], dtype=object)

In [107]:
df = df.drop_duplicates(subset=["offender_race", "offender_gender", "item_number"])

In [108]:
datetimes = (df.occurred_date_time.str.replace(r"(\w{4})-(\w{1,2})-(\w{1,2})", r"\2/\3/\1").str.extract(r"(\w{2}):(\w{2}):?(\w{2})?"))
df.loc[:, "occurred_datetime"] = datetimes[0]

C:\Users\PC\AppData\Local\Temp\ipykernel_20232\2673563878.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  datetimes = (df.occurred_date_time.str.replace(r"(\w{4})-(\w{1,2})-(\w{1,2})", r"\2/\3/\1").str.extract(r"(\w{2}):(\w{2}):?(\w{2})?"))


In [109]:
df = df[df.occurred_datetime.fillna("").str.contains("22")]
df.groupby(["occurred_datetime", "district"]).offender_race.value_counts()

occurred_datetime  district  offender_race
22                 1         BLACK            17
                             UNKNOWN           6
                             WHITE             3
                   2         BLACK            14
                             UNKNOWN           4
                             WHITE             2
                   3         BLACK            24
                             UNKNOWN          10
                             HISPANIC          1
                             WHITE             1
                   4         BLACK            16
                             UNKNOWN           2
                             WHITE             1
                   5         BLACK            11
                             UNKNOWN          10
                             WHITE             1
                   6         BLACK            12
                             UNKNOWN           7
                             WHITE             1
                   7      

In [110]:
df.shape

(387, 28)

In [111]:
requested = df[df.footage_requested.isin(["1"])]